In [1]:
!pip install openai

In [ ]:
# Create the OpenAI client
from openai import OpenAI
from os import environ

client = OpenAI(api_key=environ['OPENAI'])

# Create the request
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
   {"role": "user", "content": "I have these notes with book titles and authors: New releases this week! The Beholders by Hester Musson, The Mystery Guest by Nita Prose. Please organize the titles and authors in a json file."}
  ],
  # Specify the response format
  response_format = {'type':'json_object'}
)

# Print the response
print(response.choices[0].message.content)

In [ ]:
message = {'role': 'user',
 'content': 'Here are some made-up addresses and company names, write them in json format. PurpleLabs Solutions, 123 Main Street, Suite 100, Anytown, USA. InnovateNow Enterprises, 789 Oak Avenue, Suite 300, Innovation City, USA. PeakPerformance Inc., 456 Elm Street, Suite 200, Dreamville, USA'}
# Use the try statement
from openai import AuthenticationError

try:
    response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[message])
    # Print the response
    print(response.choices[0].message.content)
# Use the except statement
except AuthenticationError as e:
    print("Please double check your authentication key and try again, the one provided is not valid.")

In [ ]:
# Import the tenacity library
from tenacity import retry,wait_random_exponential, stop_after_attempt


# Add the appropriate parameters to the decorator
@retry(wait=wait_random_exponential(min=5,max=40), stop=stop_after_attempt(4))
def get_response(model, message):
    response = client.chat.completions.create(
      model=model,
      messages=[message]
    )
    return response.choices[0].message.content
print(get_response("gpt-4o-mini", {"role": "user", "content": "List ten holiday destinations."}))

In [ ]:
measurements = [5.2, 6.3, 3.7]

messages = []
# Provide a system message and user messages to send the batch
messages.append({'role':'system','content':'Generate conversion from kilometers to miles of the given measurement requests in batches and present the results as a table of both the original and converted measurements.'})
# Append measurements to the message
[messages.append({'role':'user','content': f'{i}'}) for i in measurements]

response = get_response(messages)
print(response)

In [ ]:
import tiktoken
input_message = {"role": "user", "content": "I'd like to buy a shirt and a jacket. Can you suggest two color pairings for these items?"}

# Use tiktoken to create the encoding for your model
encoding = tiktoken.encoding_for_model('gpt-4o-mini')
# Check for the number of tokens
num_tokens = len(encoding.encode(input_message['content']))

# Run the chat completions function and print the response
if num_tokens <= 100:
    response = client.chat.completions.create(model="gpt-4o-mini", messages=[input_message])
    print(response.choices[0].message.content)
else:
    print("Message exceeds token limit")

In [ ]:
message_listing = [{'role': 'system',
  'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."},
 {'role': 'user',
  'content': 'Step into this beautiful two-story, single-family home located in Springfield, USA, priced at $350,000. This charming property features 4 bedrooms, 2.5 bathrooms, a spacious living room with a cozy fireplace, a modern kitchen with stainless steel appliances, and a large backyard perfect for family gatherings. The master bedroom includes an en-suite bathroom and a walk-in closet. Enjoy the convenience of an attached two-car garage and a recently updated HVAC system. Located near top-rated schools, parks, and shopping centers, this home is ideal for families looking for comfort and convenience.'}]

function_definition = [{'type': 'function',
  'function': {'name': 'real_estate_info',
   'description': 'Get the information about homes for sale from the body of the input text',
   'parameters': {'type': 'object',
    'properties': {'home type': {'type': 'string', 'description': 'Home type'},
     'location': {'type': 'string', 'description': 'Location'},
     'price': {'type': 'integer', 'description': 'Price'},
     'bedrooms': {'type': 'integer', 'description': 'Number of bedrooms'}}}}}]


response= client.chat.completions.create(
    model="gpt-4o-mini",
    # Add the message
    messages=message_listing,
    # Add your function definition
    tools = function_definition
)

# Print the response
print(response.choices[0].message.tool_calls[0].function.arguments)

In [ ]:
# Define the function parameter type
function_definition[0]['function']['parameters']['type'] = 'object'

# Define the function properties
function_definition[0]['function']['parameters']['properties'] = {'title':{'type':'string','description':'Title of the paper'},'year':{'type':'string','description':'Year of the publication'}}

response = get_response(messages, function_definition)
print(response)

In [ ]:
response = get_response(messages, function_definition)

# Define the function to extract the data dictionary
def extract_dictionary(response):
  return response.choices[0].message.tool_calls[0].function.arguments

# Print the data dictionary
print(extract_dictionary(response))

In [ ]:
response= client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages,
    # Add the function definition
    tools=function_definition,
    # Specify the function to be called for the response
    tool_choice={'type':'function','function':{'name':'extract_review_info'}}
)

# Print the response
print(response.choices[0].message.tool_calls[0].function.arguments)

In [ ]:

# Define the function to pass to tools
function_definition = [{"type": 'function',
                        'function' : {"name": 'get_airport_info',
                                'description': 'This function returns information on a matching airport code',
                                'parameters': {"type": 'object', 'properties': {"airport_code": {'type':'string','description':'Returns the airport information given the aiport code'}, } }, 
                                "result": {'type':'string'}}}]

response = get_response(function_definition)
print(response)

In [ ]:

# Call the Chat Completions endpoint 
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {'role':'system','content':'You are in the aviation space now and you need to extract the corresponding airport code based on the user input.'},
    {"role": 'user', "content": "I'm planning to land a plane in JFK airport in New York and would like to have the corresponding information."}],
  tools=function_definition)

print(response)

In [ ]:
if response.choices[0].finish_reason=='tool_calls':
  function_call = response.choices[0].message.tool_calls[0].function
  # Check function name
  if function_call.name == 'get_airport_info':
    # Extract airport code
    code = json.loads(function_call.arguments)['airport code']
    airport_info = get_airport_info(code)
    print(airport_info)
  else:
    print("Apologies, I couldn't find any airport.")
else: 
  print("I am sorry, but I could not understand your request.")

In [ ]:
message = "Can you show some example sentences in the past tense in French?"

# Use the moderation API
moderation_response = (client.moderations.create(input=message))

# Print the response
print(moderation_response.results[0].categories.violence)

In [ ]:
import uuid

# Generate a unique ID
unique_id = str(uuid.uuid4())

response = client.chat.completions.create(  
  model="gpt-4o-mini", 
  messages=messages,
# Pass a user identification key
  user = unique_id
)

print(response.choices[0].message.content)